In [1]:
import pandas as pd
import os
import torch
from utils.parse import parse_yaml
import json
import numpy as np
from models.resnet50v2_sn_model import Model
from tqdm import tqdm
from PIL import Image
from datetime import datetime
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
tqdm.monitor_interval = 0

# load config

In [2]:
config = parse_yaml()
config = config['whole']
config['logger']=False
config['use_tensorboard']=False

# load model

In [3]:
model = Model(config)
model.load('ckpt/2018Oct16-183555/17.pth')

>>> Load model completion.


# evaluation

In [4]:
res_json = []
path = 'data/ai_challenger_pdr2018_testA_20180905/AgriculturalDisease_testA/images'
files = os.listdir(path)
with tqdm(total=len(files)) as pbar:
    for img_id in files:
        img_path = os.path.join(path, img_id)
        img = Image.open(img_path)
        compose = transforms.Compose([
            transforms.Resize([224, 224]),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.39883098, 0.48478258, 0.46141425],
                std=[0.21294028, 0.17201832, 0.1934495])
        ])
        img_tensor = compose(img)
        img_tensor = img_tensor[None, :, :, :]
        res = model.test(img_tensor).cpu().data.numpy()
        pred_class = int(np.argmax(res, axis=1)[0])
        res_json.append({"image_id": img_id, "disease_class": pred_class})
        pbar.update(1)
        pbar.set_description(
            "Processing {}, Prediction {}".format(img_id, pred_class))

Processing ab916331-4f8d-4993-a333-85503ad30362___FAM_B.Msls 4165.JPG, Prediction 21: 100%|██████████| 4959/4959 [05:42<00:00, 14.50it/s]              


# save json

In [5]:
run_timestamp = datetime.now().strftime("%Y%b%d-%H%M%S")
with open('submit/{}.json'.format(run_timestamp), 'w') as json_file:
    json.dump(res_json, json_file)